<a href="https://colab.research.google.com/github/TechSlinger/TechSlinger.github.io/blob/main/TextGenerationResponse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
!pip install -q cassandra-driver
!pip install -q cassio>=0.1.1
!pip install -q gradientai --upgrade
!pip install  llama-index
!pip install -q pypdf
!pip install -q tiktoken==0.4.0

In [20]:
!pip install llama-index-llms-gradient
!pip install llama-index-embeddings-gradient
!pip install llama-index-vector-stores-cassandra

In [8]:
import os
import json
from google.colab import userdata

os.environ['GRADIENT_ACCESS_TOKEN'] ='W9Mkj70Q01rM7bQJuqS7EhrYsx1wNWhb'
os.environ['GRADIENT_WORKSPACE_ID'] = 'c8abedd7-3556-49a9-9c28-2ad60d3f6c1d_workspace'

In [21]:
from cassandra.auth import PlainTextAuthProvider
from cassandra.cluster import Cluster
from llama_index.core import ServiceContext
from llama_index.core import set_global_service_context
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, StorageContext
from llama_index.embeddings.gradient import GradientEmbedding
from llama_index.llms.gradient import GradientBaseModelLLM
from llama_index.vector_stores.cassandra import CassandraVectorStore

In [23]:
import cassandra
print (cassandra.__version__)

3.29.0


In [24]:

# This secure connect bundle is autogenerated when you donwload your SCB,
# if yours is different update the file name below
cloud_config= {
  'secure_connect_bundle': 'secure-connect-textai.zip'
}

# This token json file is autogenerated when you donwload your token,
# if yours is different update the file name below
with open("textAI-token.json") as f:
    secrets = json.load(f)

CLIENT_ID = secrets["clientId"]
CLIENT_SECRET = secrets["secret"]

auth_provider = PlainTextAuthProvider(CLIENT_ID, CLIENT_SECRET)
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
session = cluster.connect()

row = session.execute("select release_version from system.local").one()
if row:
  print(row[0])
else:
  print("An error occurred.")

ERROR:cassandra.connection:Closing connection <AsyncoreConnection(133806440640960) 9fb6f02e-7507-4b04-b227-5194f7ffde19-us-east1.db.astra.datastax.com:29042:c39a5bc8-1a0c-4e01-ab3a-883b2afa0063> due to protocol error: Error from server: code=000a [Protocol error] message="Beta version of the protocol used (5/v5-beta), but USE_BETA flag is unset"


4.0.0.6816


In [25]:
llm = GradientBaseModelLLM(
    base_model_slug="llama2-7b-chat",
    max_tokens=400,
)

In [26]:
embed_model = GradientEmbedding(
    gradient_access_token = os.environ["GRADIENT_ACCESS_TOKEN"],
    gradient_workspace_id = os.environ["GRADIENT_WORKSPACE_ID"],
    gradient_model_slug="bge-large",
)


In [27]:
service_context = ServiceContext.from_defaults(
    llm = llm,
    embed_model = embed_model,
    chunk_size=256,
)

set_global_service_context(service_context)

<ipython-input-27-944266470549>:1: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(


In [28]:
documents = SimpleDirectoryReader("/content/Documents").load_data()
print(f"Loaded {len(documents)} document(s).")


Loaded 20 document(s).


In [29]:
index = VectorStoreIndex.from_documents(documents,
                                   service_context=service_context)
query_engine = index.as_query_engine()

In [30]:
response = query_engine.query("What is the purpose of Google's operational flood forecasting system?")
print(response)

 Based on the provided context information, the purpose of Google's operational flood forecasting system is to provide accurate real-time flood warnings to agencies and the public with a focus on riverine floods in large, gauged rivers.
